# Important Word Detection

1. Load Dataset and pick 60 Documents for now
2. Do sentence splitting
3. Leave One Out (LOO)

4. Find important Word through Prediciton



In [1]:
import pandas as pd
import json
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize
import os

import utils.text_processing as tp

## 1. Load Data and pick first 60 examples for now

In [2]:
data = tp.load_jsonline(os.path.join('data', 'items_reviews_18.jl'), 59)

In [3]:
data[0]

{'target_id': 3321611,
 'source_id': 277605655,
 'title': 'This place is incredible!',
 'text': 'I visited this b&b during a short trip to ride the famous Belgian pavé and it was perfect. The owners were really lovely people, the room was very comfortable and the breakfast was a delicious feast- ideal for big days out on the bicycle! It is in a really good location for riding or driving into Oudenaarde (approx 10 mins) and there are some brilliant restaurants close by. I cannot recommend this place enough!',
 'user_rating': 5,
 'lang': 'en',
 '_type': 'TripAdvisorHotelReviewItem'}

## 2. Sentence Splitting
- have list with text items
- have list with splitted sentences

In [4]:
# List of Text form Reviews
sentences = []
for obj in data:
    sentences.append(obj["text"])

In [5]:
# List of single Sentences found in all available Text
sentence_list = sent_tokenize(". ".join(sentences))

In [6]:
print('Lenght sentences:', len(sentences), '\n', 'Length sentence_list:', len(sentence_list))

Lenght sentences: 60 
 Length sentence_list: 435


## 3. Leave One Out (LOO)

In [7]:
# List of Lists of tokenized sentences
tok_sentences = []
i = 0
for sentence in sentence_list:
    tok_sentences.append(sentence_list[i].split(' '))
    i += 1

len(tok_sentences)

435

In [8]:
# go over the list of tokens in a sentence
# and drop each word after the other
# go over sentences in list of tokenized sentences
sentence_packages = []
for sent in range(len(tok_sentences)):
    original_sentence = tp.detokenize(tok_sentences[sent])
    modified_sentences = []
# go over token in sentence
    for token in range(len(tok_sentences[sent])):
        tok_mod_sentence = tp.get_token_dropped_sentence_at_pos(tok_sentences[sent], token)
        modified_sentences.append((tok_sentences[sent][token], tp.detokenize(tok_mod_sentence)))
    sentence_packages.append(
        {
            'original_sentence':original_sentence,
            'modified_sentences':modified_sentences
        }        
    )

Juhuuuuu :D

# 4. Predict with BERT for Sentiment Classification

In [9]:
import torch
import random
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model.eval();
len(tokenizer.vocab)

105879

In [10]:
important_words = []

for package in sentence_packages:    
    original_sentence = package['original_sentence']
    # print('new package: ' + original_sentence)

    original_result = tp.predict_sentiment(model, tokenizer, original_sentence)
    highest_relative = 0
    highest_relative_word = None

    for item in package['modified_sentences']:
        word = item[0]
        sentence = item[1]
        modified_result = tp.predict_sentiment(model, tokenizer, sentence)
        relative = abs(original_result - modified_result)
# OBACHT >=
        if relative >= highest_relative:
            highest_relative = relative
            highest_relative_word = word
            
    important_words.append(highest_relative_word)

assert(len(important_words)==len(sentence_packages))    
# print(important_words)

KeyboardInterrupt: 

In [ ]:
%store important_words
%store sentence_packages

In [ ]:
len(important_words)